# DURATION-NEUTRAL DTS-TARGET STRESS TEST

In [2]:
import polars as pl
import numpy as np
from scipy.stats import norm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Load trading results
results = pl.read_excel("Data/DTSResults.xlsx").with_columns((pl.col('total_return') / pl.col('portfolio_cost')).alias('pct_return'))
results = results.with_columns(np.log(1 + pl.col('pct_return')).alias('log_return'))

results

coupon_income,capital_gains,hedge_cost,portfolio_cost,total_return,excess_return,date,bucket,rf_rate,pct_return,log_return
f64,f64,f64,f64,f64,f64,date,i64,f64,f64,f64
0.0,0.0,2.402919,100.721128,0.0,0.0,2022-05-18,1,0.001,0.0,0.0
0.0,0.0,5.351416,98.650057,0.0,0.0,2022-05-18,2,0.001,0.0,0.0
0.0,0.0,14.502309,94.387951,0.0,0.0,2022-05-18,3,0.001,0.0,0.0
2.082298,0.15425,-0.265294,101.12161,2.557952,-0.576818,2022-05-25,1,0.001,0.025296,0.024981
2.886862,0.112217,-0.915174,99.033699,4.148431,0.187083,2022-05-25,2,0.001,0.041889,0.041035
…,…,…,…,…,…,…,…,…,…,…
3.614204,-0.033451,-12.319218,97.506407,20.821681,-55.233317,2024-11-06,2,0.02,0.213542,0.193543
3.013474,-1.552656,29.062552,70.247179,-19.271047,-64.229242,2024-11-06,3,0.02,-0.274332,-0.320663
5.330372,0.057681,-7.389142,96.193708,15.635603,-36.308999,2024-11-13,1,0.02,0.162543,0.15061


- calculate mu and sigma using EWMA

In [3]:
result = results.tail(52 * 3)
result = result.select('bucket', 'log_return', 'date')
final = result.pivot(columns = 'bucket', index = 'date', values = 'log_return')

final

date,1,2,3
date,f64,f64,f64
2023-11-08,-0.031055,0.086246,-0.188572
2023-11-15,0.114765,-0.039418,0.204301
2023-11-22,-0.050544,0.09244,-0.218956
2023-11-29,0.113529,-0.062497,0.188643
2023-12-06,-0.053549,0.064607,-0.205046
…,…,…,…
2024-10-16,0.128193,-0.076667,0.162838
2024-10-23,-0.038053,0.16511,-0.334145
2024-10-30,0.129295,-0.086185,0.141702


In [4]:
def EWMA(mu, cov, X):
        cov = theta * cov  + (1 - theta) * np.outer((X - mu), (X - mu).T)
        mu = lambd * mu + (1 - lambd) * X
    
        return mu, cov
    
def initialize():
    mu = np.array([0, 0, 0])
    cov = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    
    for i in range(len(final)):
        X = np.array([final['1'][i], final['2'][i], final['3'][i]])
        mu, cov = EWMA(mu, cov, X)

    return mu, cov, X

lambd = 0.97
theta = 0.97
alpha = 0.95
K = 4
port_value = 1000000

In [5]:
def simulation_shock2(mu, cov, X, shock = True):

    if shock == True:
        mu2 = mu[1]
        var2 = cov[1][1]
        X[1] = mu2 - 5 * np.sqrt(var2)
    
        std_devs = np.sqrt(np.diag(cov))
        rho = cov / np.outer(std_devs, std_devs)
        
        mu1 = mu[0] + (rho[0][1] * np.sqrt(cov[0][0]) / np.sqrt(cov[1][1])) * (X[1] - mu2)
        var1 = cov[0][0] * (1 - rho[0][1]**2)
        
        X[0] = mu1 - 5 * np.sqrt(var1)
    
        mu3 = mu[2] + (rho[1][2] * np.sqrt(cov[2][2]) / np.sqrt(cov[1][1])) * (X[1] - mu2)
        var3 = cov[2][2] * (1 - rho[1][2]**2)
        
        X[2] = mu3 - 5 * np.sqrt(var3)

    else:
        X[0] = mu[0]
        X[1] = mu[1]
        X[2] = mu[2]
        

    # Now the mu and cov are at t + 2 delta
    mu, cov = EWMA(mu, cov, X)

    sum_X = X

    for i in range(2, K + 1):
        X_samp = np.random.multivariate_normal(mu, cov)
        sum_X += X_samp
        if i != K:
            mu, cov = EWMA(mu, cov, X)

    return -port_value * sum_X

def simulation_shock1(mu, cov, X, shock = True):

    if shock == True:
        mu1 = mu[0]
        var1 = cov[0][0]
        X[0] = mu1 - 5 * np.sqrt(var1)
    
        std_devs = np.sqrt(np.diag(cov))
        rho = cov / np.outer(std_devs, std_devs)
        
        mu2 = mu[1] + (rho[0][1] * np.sqrt(cov[1][1]) / np.sqrt(cov[0][0])) * (X[0] - mu1)
        var2 = cov[1][1] * (1 - rho[0][1]**2)
        
        X[1] = mu2 - 5 * np.sqrt(var2)
    
        mu3 = mu[2] + (rho[0][2] * np.sqrt(cov[2][2]) / np.sqrt(cov[0][0])) * (X[0] - mu1)
        var3 = cov[2][2] * (1 - rho[0][2]**2)
        
        X[2] = mu3 - 5 * np.sqrt(var3)

    else:
        X[0] = mu[0]
        X[1] = mu[1]
        X[2] = mu[2]
        

    # Now the mu and cov are at t + 2 delta
    mu, cov = EWMA(mu, cov, X)

    sum_X = X

    for i in range(2, K + 1):
        X_samp = np.random.multivariate_normal(mu, cov)
        sum_X += X_samp
        if i != K:
            mu, cov = EWMA(mu, cov, X)

    return -port_value * sum_X

def simulation_shock3(mu, cov, X, shock = True):

    if shock == True:
        mu3 = mu[2]
        var3 = cov[2][2]
        X[2] = mu3 - 5 * np.sqrt(var3)
    
        std_devs = np.sqrt(np.diag(cov))
        rho = cov / np.outer(std_devs, std_devs)
        
        mu2 = mu[1] + (rho[2][1] * np.sqrt(cov[1][1]) / np.sqrt(cov[2][2])) * (X[2] - mu3)
        var2 = cov[1][1] * (1 - rho[2][1]**2)
        
        X[1] = mu2 - 5 * np.sqrt(var2)
    
        mu1 = mu[0] + (rho[0][2] * np.sqrt(cov[0][0]) / np.sqrt(cov[2][2])) * (X[2] - mu3)
        var1 = cov[0][0] * (1 - rho[0][2]**2)
        
        X[0] = mu1 - 5 * np.sqrt(var1)

    else:
        X[0] = mu[0]
        X[1] = mu[1]
        X[2] = mu[2]
        

    # Now the mu and cov are at t + 2 delta
    mu, cov = EWMA(mu, cov, X)

    sum_X = X

    for i in range(2, K + 1):
        X_samp = np.random.multivariate_normal(mu, cov)
        sum_X += X_samp
        if i != K:
            mu, cov = EWMA(mu, cov, X)

    return -port_value * sum_X

In [6]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock1(mu, cov, X, True)
    sums.append(np.dot(weights, sum.copy()))

losses_shock1 = pl.DataFrame(sums).rename({'column_0' : 'Low'})

In [7]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock2(mu, cov, X, True)
    sums.append(np.dot(weights, sum.copy()))

losses_shock2 = pl.DataFrame(sums).rename({'column_0' : 'Mid'})

In [8]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock3(mu, cov, X, True)
    sums.append(np.dot(weights, sum.copy()))

losses_shock3 = pl.DataFrame(sums).rename({'column_0' : 'High'})

In [9]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock1(mu, cov, X, False)
    sums.append(np.dot(weights, sum.copy()))

losses_1 = pl.DataFrame(sums).rename({'column_0' : 'Low'})

In [10]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock2(mu, cov, X, False)
    sums.append(np.dot(weights, sum.copy()))

losses_2 = pl.DataFrame(sums).rename({'column_0' : 'Mid'})

In [11]:
sums = []
weights = np.array([0.6, 0.3, 0.1])
mu, cov, X = initialize()

for i in range(50000):
    sum = simulation_shock3(mu, cov, X, False)
    sums.append(np.dot(weights, sum.copy()))

losses_3 = pl.DataFrame(sums).rename({'column_0' : 'High'})

In [12]:
avg_shock1 = losses_shock1.mean().to_pandas().rename({0 : 'shock'})
avg_shock2 = losses_shock2.mean().to_pandas().rename({0 : 'shock'})
avg_shock3 = losses_shock3.mean().to_pandas().rename({0 : 'shock'})

avg_1 = losses_1.mean().to_pandas().rename({0 : 'no shock'})
avg_2 = losses_2.mean().to_pandas().rename({0 : 'no shock'})
avg_3 = losses_3.mean().to_pandas().rename({0 : 'no shock'})

In [17]:
round(pd.concat([avg_shock1, avg_1]).join(pd.concat([avg_shock2, avg_2])).join(pd.concat([avg_shock3, avg_3])).transpose(), 2)

,shock,no shock
Low,372407.77,-120550.55
Mid,57549.83,-119301.39
High,389093.93,-119839.89


In [19]:
round(pd.concat([avg_shock1, avg_1]).join(pd.concat([avg_shock2, avg_2])).join(pd.concat([avg_shock3, avg_3])).transpose(), 2).mean()

shock       273017.176667
no shock   -119897.276667
dtype: float64

In [14]:
var_shock1 = losses_shock1.quantile(alpha).to_pandas().rename({0 : 'shock'})
var_shock2 = losses_shock2.quantile(alpha).to_pandas().rename({0 : 'shock'})
var_shock3 = losses_shock3.quantile(alpha).to_pandas().rename({0 : 'shock'})

var_1 = losses_1.quantile(alpha).to_pandas().rename({0 : 'no shock'})
var_2 = losses_2.quantile(alpha).to_pandas().rename({0 : 'no shock'})
var_3 = losses_3.quantile(alpha).to_pandas().rename({0 : 'no shock'})

In [18]:
round(pd.concat([var_shock1, var_1]).join(pd.concat([var_shock2, var_2])).join(pd.concat([var_shock3, var_3])).transpose(), 2)

,shock,no shock
Low,718549.20,33474.31
Mid,245717.02,34231.51
High,740607.31,33230.76


In [20]:
round(pd.concat([var_shock1, var_1]).join(pd.concat([var_shock2, var_2])).join(pd.concat([var_shock3, var_3])).transpose(), 2).mean()

shock       568291.176667
no shock     33645.526667
dtype: float64